<h2><a href="https://leetcode.com/problems/find-the-subtasks-that-did-not-execute">1914. Find the Subtasks That Did Not Execute</a></h2><h3>Hard</h3><hr><p>Table: <code>Tasks</code></p>

<pre>
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| task_id        | int     |
| subtasks_count | int     |
+----------------+---------+
task_id is the column with unique values for this table.
Each row in this table indicates that task_id was divided into subtasks_count subtasks labeled from 1 to subtasks_count.
It is guaranteed that 2 &lt;= subtasks_count &lt;= 20.
</pre>

<p>&nbsp;</p>

<p>Table: <code>Executed</code></p>

<pre>
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| task_id       | int     |
| subtask_id    | int     |
+---------------+---------+
(task_id, subtask_id) is the combination of columns with unique values for this table.
Each row in this table indicates that for the task task_id, the subtask with ID subtask_id was executed successfully.
It is <strong>guaranteed</strong> that subtask_id &lt;= subtasks_count for each task_id.</pre>

<p>&nbsp;</p>

<p>Write a solution&nbsp;to report the IDs of the missing subtasks for each <code>task_id</code>.</p>

<p>Return the result table in <strong>any order</strong>.</p>

<p>The result format is in the following example.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>

<pre>
<strong>Input:</strong> 
Tasks table:
+---------+----------------+
| task_id | subtasks_count |
+---------+----------------+
| 1       | 3              |
| 2       | 2              |
| 3       | 4              |
+---------+----------------+
Executed table:
+---------+------------+
| task_id | subtask_id |
+---------+------------+
| 1       | 2          |
| 3       | 1          |
| 3       | 2          |
| 3       | 3          |
| 3       | 4          |
+---------+------------+
<strong>Output:</strong> 
+---------+------------+
| task_id | subtask_id |
+---------+------------+
| 1       | 1          |
| 1       | 3          |
| 2       | 1          |
| 2       | 2          |
+---------+------------+
<strong>Explanation:</strong> 
Task 1 was divided into 3 subtasks (1, 2, 3). Only subtask 2 was executed successfully, so we include (1, 1) and (1, 3) in the answer.
Task 2 was divided into 2 subtasks (1, 2). No subtask was executed successfully, so we include (2, 1) and (2, 2) in the answer.
Task 3 was divided into 4 subtasks (1, 2, 3, 4). All of the subtasks were executed successfully.
</pre>

In [1]:
from pyspark.sql import SparkSession

# Initialize a SparkSession
spark = SparkSession.builder.appName("SQLPractice").getOrCreate()

# Create the Tasks DataFrame
tasks_data = [
    (1, 3),
    (2, 2),
    (3, 4)
]
tasks_columns = ["task_id", "subtasks_count"]
tasks_df = spark.createDataFrame(tasks_data, schema=tasks_columns)

# Create the Executed DataFrame
executed_data = [
    (1, 2),
    (3, 1),
    (3, 2),
    (3, 3),
    (3, 4)
]
executed_columns = ["task_id", "subtask_id"]
executed_df = spark.createDataFrame(executed_data, schema=executed_columns)

# Register the DataFrames as temporary views for SQL queries
tasks_df.createOrReplaceTempView("Tasks")
executed_df.createOrReplaceTempView("Executed")

tasks_df.show()
executed_df.show()


25/02/23 16:23:23 WARN Utils: Your hostname, Nahils-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.103 instead (on interface en0)
25/02/23 16:23:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/23 16:23:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/23 16:23:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+-------+--------------+
|task_id|subtasks_count|
+-------+--------------+
|      1|             3|
|      2|             2|
|      3|             4|
+-------+--------------+

+-------+----------+
|task_id|subtask_id|
+-------+----------+
|      1|         2|
|      3|         1|
|      3|         2|
|      3|         3|
|      3|         4|
+-------+----------+



In [10]:
# Use SQL to create a range of values from 1 to 20
spark.sql("""
    WITH subtask_id_table AS (
        SELECT explode(array(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20)) AS subtask_id
    ),
          
    all_tasks_table AS (
        SELECT t.task_id, st.subtask_id
            FROM Tasks t
            INNER JOIN subtask_id_table st ON st.subtask_id <= t.subtasks_count 
    )
          
    SELECT * FROM all_tasks_table
    EXCEPT
    SELECT * FROM Executed
""").show()


+-------+----------+
|task_id|subtask_id|
+-------+----------+
|      1|         3|
|      1|         1|
|      2|         2|
|      2|         1|
+-------+----------+



In [11]:
spark.stop()